In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
!conda install -c anaconda beautifulsoup4
import pandas as pd
import bs4 as bsoup
from bs4 import BeautifulSoup
import requests
import numpy as np
!conda install -c conda-forge folium=0.5.0
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge geocoder
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [2]:
url='http://techslides.com/list-of-countries-and-capitals'
page=requests.get(url)
soup2=BeautifulSoup(page.content, 'html.parser')
#tb=soup2.find('table', class_='wikitable')
#for link in tb.find_all('b'):
    #name=link.find('a')
    #print(name.get_text('title'))
table=soup2.find_all('table')[0]
df=pd.read_html(str(table))[0]
df.columns=df.iloc[0]
df=df.drop(df.index[0])
df=df.reset_index(drop=True)
df.head()

,Country Name,Capital Name,Capital Latitude,Capital Longitude,Country Code,Continent Name
0,Afghanistan,Kabul,34.51666667,69.183333,AF,Asia
1,Aland Islands,Mariehamn,60.116667,19.9,AX,Europe
2,Albania,Tirana,41.31666667,19.816667,AL,Europe
3,Algeria,Algiers,36.75,3.05,DZ,Africa
4,American Samoa,Pago Pago,-14.26666667,-170.7,AS,Australia


In [3]:
CLIENT_ID = 'WXTQZPHF0DVEZSX2JZF1SM2W42IZ4GKYLTDD3NXT5RH5MFC0' # your Foursquare ID
CLIENT_SECRET = 'OYJMUIILZBFG0FKRXXV3AQSA0NHH0DBUQ0DMYJKPWWPAINUJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
manhattan_data=df.iloc[0:2]
manhattan_data

,Country Name,Capital Name,Capital Latitude,Capital Longitude,Country Code,Continent Name
0,Afghanistan,Kabul,34.51666667,69.183333,AF,Asia
1,Aland Islands,Mariehamn,60.116667,19.9,AX,Europe


In [4]:
LIMIT=500
radius=1000
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Capital Name'],
                                   latitudes=manhattan_data['Capital Latitude'],
                                   longitudes=manhattan_data['Capital Longitude']
                                  )

manhattan_venues

Kabul
Mariehamn


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kabul,34.51666667,69.183333,Kabul Serena Hotel,34.520398,69.177970,Hotel
1,Kabul,34.51666667,69.183333,Morād Khānī,34.517470,69.181520,Neighborhood
2,Kabul,34.51666667,69.183333,جاده نادر پشتون,34.518645,69.180619,Flea Market
3,Kabul,34.51666667,69.183333,Kabul City,34.516647,69.187922,Farm
4,Kabul,34.51666667,69.183333,Cafe Zarnegar,34.520319,69.178185,Afghan Restaurant
5,Kabul,34.51666667,69.183333,Kabul River,34.516935,69.174385,River
6,Mariehamn,60.116667,19.9,S-market,60.113546,19.914325,Supermarket
7,Mariehamn,60.116667,19.9,Taurus,60.112186,19.893957,Burrito Place
8,Mariehamn,60.116667,19.9,Feja,60.112267,19.912491,Paper / Office Supplies Store
9,Mariehamn,60.116667,19.9,Crossfit North AX,60.111083,19.911427,Gym


In [6]:
# one hot encoding
world_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
world_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [world_onehot.columns[-1]] + list(world_onehot.columns[:-1])
world_onehot = world_onehot[fixed_columns]

world_onehot.head()

,Supermarket,Afghan Restaurant,Burrito Place,Farm,Flea Market,Gym,Hotel,Lake,Neighborhood,Paper / Office Supplies Store,River
0,0,0,0,0,0,0,1,0,Kabul,0,0
1,0,0,0,0,0,0,0,0,Kabul,0,0
2,0,0,0,0,1,0,0,0,Kabul,0,0
3,0,0,0,1,0,0,0,0,Kabul,0,0
4,0,1,0,0,0,0,0,0,Kabul,0,0


In [7]:
manhattan_grouped = world_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Supermarket,Afghan Restaurant,Burrito Place,Farm,Flea Market,Gym,Hotel,Lake,Paper / Office Supplies Store,River
0,Kabul,0.0,0.166667,0.0,0.166667,0.166667,0.0,0.166667,0.0,0.0,0.166667
1,Mariehamn,0.2,0.000000,0.2,0.000000,0.000000,0.2,0.000000,0.2,0.2,0.000000


In [8]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Kabul----
               venue  freq
0  Afghan Restaurant  0.17
1               Farm  0.17
2        Flea Market  0.17
3              Hotel  0.17
4              River  0.17


----Mariehamn----
                           venue  freq
0                    Supermarket   0.2
1                  Burrito Place   0.2
2                            Gym   0.2
3                           Lake   0.2
4  Paper / Office Supplies Store   0.2




In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kabul,River,Hotel,Flea Market,Farm,Afghan Restaurant,Paper / Office Supplies Store,Lake,Gym,Burrito Place,Supermarket
1,Mariehamn,Paper / Office Supplies Store,Lake,Gym,Burrito Place,Supermarket,River,Hotel,Flea Market,Farm,Afghan Restaurant


In [11]:
# set number of clusters
kclusters = 1

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0], dtype=int32)

In [12]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Capital Name')
manhattan_merged['Cluster Labels']=manhattan_merged['Cluster Labels'].astype(int)
manhattan_merged['Capital Latitude']=manhattan_merged['Capital Latitude'].astype(float)
manhattan_merged['Capital Longitude']=manhattan_merged['Capital Longitude'].astype(float)

manhattan_merged.head() # check the last columns!

,Country Name,Capital Name,Capital Latitude,Capital Longitude,Country Code,Continent Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Afghanistan,Kabul,34.516667,69.183333,AF,Asia,0,River,Hotel,Flea Market,Farm,Afghan Restaurant,Paper / Office Supplies Store,Lake,Gym,Burrito Place,Supermarket
1,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe,0,Paper / Office Supplies Store,Lake,Gym,Burrito Place,Supermarket,River,Hotel,Flea Market,Farm,Afghan Restaurant


In [16]:
%matplotlib notebokk
# create map
map_clusters = folium.Map(location=[0,0], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Capital Latitude'], manhattan_merged['Capital Longitude'], manhattan_merged['Capital Name'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'notebokk'